In [96]:
import pandas as pd
import os
import matplotlib.pyplot as pyplt
import seaborn as sns
sns.set()
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [97]:
coord_dir = '../capitol riots/index_20221102'
hashtag_output_file = 'coord_user_hashtag_final_20221102.csv'
mentions_output_file = 'coord_user_mentions_final_20221102.csv'
urls_output_file = 'coord_user_urls_final_20221102.csv'

bothunter_folder = '../bothunter output/capitolriots'
bothunter_file = 'botoutput.csv'
bot_file = os.path.join(bothunter_folder, bothunter_file)

In [98]:
df_bots = pd.read_csv(bot_file)
df_bots = df_bots[df_bots['probability'] != 'probability']
df_bots['IS_BOT'] = df_bots['probability'].apply(lambda x: True if float(x) > 0.7 else False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [99]:
df_hashtag_pairs = pd.read_csv(os.path.join(coord_dir, 'coord_user_simple_multiply_hashtag_20221102.csv'))
df_mentions_pairs = pd.read_csv(os.path.join(coord_dir, 'coord_user_simple_multiply_mentions_20221102.csv'))
df_urls_pairs = pd.read_csv(os.path.join(coord_dir, 'coord_user_simple_multiply_urls_20221102.csv'))

In [100]:
ora_export_dir = '../capitol riots/ORA Exports'
nodeset_file_1 = 'Capitol Hashtag-network-Agent x Agent.csv'
df_hashtag = pd.read_csv(os.path.join(ora_export_dir, nodeset_file_1))

nodeset_file_2 = 'Capitol Mentions-network-Agent x Agent.csv'
df_mentions = pd.read_csv(os.path.join(ora_export_dir, nodeset_file_2))

nodeset_file_3 = 'Capitol URL-network-Agent x Agent.csv'
df_urls = pd.read_csv(os.path.join(ora_export_dir, nodeset_file_3))

In [101]:
def get_pair_coord_total(df, coord_index):
    #link_value = 0
    counter = 0
    user_coord_value = 0
    
    for idx, row in df.iterrows():
        #link_value += coord_index
        counter += 1
            
        user_coord_value += row['Link Value'] * coord_index
    
    return user_coord_value, counter

In [34]:
df_hashtag.head()

,Source Node ID,Target Node ID,Link Value
0,rumbera1055,venezuelaadiar1,106.0
1,RumberaF,rumbera1055,118.0
2,RumberaF,venezuelaadiar1,90.0
3,RumberaF,RumberanetworkF,110.0
4,DoralNewsFL,rumbera1055,118.0


In [102]:
def get_link_total(df):
    link_value = 0
    for idx, row in df.iterrows():
        link_value += row['Link Value']
    
    return link_value

In [103]:
def get_user_coord(df_pairs, df_ora, output_file):
    df_pairs_unique_users = df_pairs['user'].unique().tolist()
    
    df_user_coord_arr = []

    for user in df_pairs_unique_users:
        df_ora_subset_source = df_ora[df_ora['Source Node ID'] == user]
        df_ora_subset_target = df_ora[df_ora['Target Node ID'] == user]
        link_values = get_link_total(df_ora_subset_source) + get_link_total(df_ora_subset_target)
        
        df_subset = df_pairs[(df_pairs['user'] == user)]
        coord_index = df_subset.iloc[0]['coord_index']
        
        #coord_total, counter = get_pair_coord_total(df_subset, coord_index)

        coord_total = coord_index * link_values
        
        coord_sigmoid = sigmoid(coord_total)

        obj = {'user': user, 'coord_index': coord_total, 'coord_sigmoid': coord_sigmoid}
        df_user_coord_arr.append(obj)
        
    df_user_coord = pd.DataFrame(df_user_coord_arr)
    df_user_coord.head()
    
    df_merge = df_user_coord.merge(df_bots, how='inner', left_on='user', right_on='screen_name')
    df_merge = df_merge.drop_duplicates(subset=['user'])
    
    df_merge.to_csv(os.path.join(coord_dir, output_file), index=False)

In [33]:
df_hashtag_pairs.head()

,user,link_value_1,num_coordination,total_link_value,total_link_valuenormalized,coord_index
0,BaldeckJantzen,384.0,1,384.0,1.035113,1.035113
1,LadyIMM1,112.0,1,112.0,1.010241,1.010241
2,Toronto1,0.0,1,0.0,1.000000,1.000000
3,WenMaMa2,0.0,1,0.0,1.000000,1.000000
4,mikedeth,112.0,1,112.0,1.010241,1.010241


In [104]:
get_user_coord(df_hashtag_pairs, df_hashtag, hashtag_output_file)
get_user_coord(df_mentions_pairs, df_mentions, mentions_output_file)
get_user_coord(df_urls_pairs, df_urls, urls_output_file)

In [159]:
df_user_coord['coord_index'].describe()

count    3563.000000
mean        2.571227
std         8.751025
min         0.000000
25%         1.004608
50%         1.004608
75%         1.004608
max       434.000000
Name: coord_index, dtype: float64

In [160]:
df_user_coord['coord_index'].median()

1.0046082949308757

In [204]:
df_merge_bots = df_merge[df_merge['IS_BOT'] == True]
df_merge_humans = df_merge[df_merge['IS_BOT'] == False]

In [205]:
df_merge_bots['coord_index'].describe()

count    3572.000000
mean        7.375041
std         5.395809
min         4.000000
25%         5.025907
50%         5.025907
75%        11.398964
max       109.699482
Name: coord_index, dtype: float64

In [206]:
df_merge_humans['coord_index'].describe()

count    6677.000000
mean        7.218327
std         5.418153
min         4.000000
25%         4.000000
50%         5.025907
75%        11.398964
max       116.974093
Name: coord_index, dtype: float64